In [11]:
import cv2
import os
import numpy as np
import tensorflow as tf
from random import Random

rng = Random(1410)

In [12]:
COCO_IMAGE_PATH = '/home/pwl/Projects/tiny-tracking/COCO/val2017'

In [13]:
target_shapes = {
    'yolox': (416, 416),
}

f32_model_paths = {
    'yolox': '/home/pwl/Projects/tiny-tracking/source_models/yolox_nano/yolox_nano_ti_lite_26p1_41p8',
}

In [14]:
image_files = [f for f in os.listdir(COCO_IMAGE_PATH)]
rng.shuffle(image_files)
selected_images = image_files[:500]

def representative_dataset_gen():
    for image_name in selected_images:
        image_path = os.path.join(COCO_IMAGE_PATH, image_name)
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        resized_img = cv2.resize(img, target_shapes['yolox'])
        resized_img = np.expand_dims(resized_img, axis=0)
        resized_img = resized_img.astype(np.float32)
        yield [resized_img]

In [15]:

converter = tf.lite.TFLiteConverter.from_saved_model(f32_model_paths['yolox'])
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
tflite_quant_model = converter.convert()
open("yolox_nano_quant.tflite", "wb").write(tflite_quant_model)

2023-07-18 00:07:14.243910: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-07-18 00:07:14.243933: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-07-18 00:07:14.244103: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /home/pwl/Projects/tiny-tracking/source_models/yolox_nano/yolox_nano_ti_lite_26p1_41p8
2023-07-18 00:07:14.251311: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2023-07-18 00:07:14.251340: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /home/pwl/Projects/tiny-tracking/source_models/yolox_nano/yolox_nano_ti_lite_26p1_41p8
2023-07-18 00:07:14.264030: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-07-18 00:07:14.303185: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: /home/pwl/Project

7452792